In [ ]:
from collections import OrderedDict
from dautil import data
from dautil import ts
from dautil import plotting
import numpy as np
import bokeh.plotting as bkh_plt
from bokeh.models import HoverTool

In [ ]:
def load():
    df = data.Weather.load()['TEMP']
    return ts.groupby_year_month(df)

In [ ]:
def create_source():
    colors = plotting.sample_hex_cmap()

    month = []
    year = []
    color = []
    avg = []

    for year_month, group in load():
        month.append(ts.short_month(year_month[1]))
        year.append(str(year_month[0]))
        monthly_avg = np.nanmean(group.values)
        avg.append(monthly_avg)
        color.append(colors[min(int(abs(monthly_avg)) - 2, 8)])

    source = bkh_plt.ColumnDataSource(
        data=dict(month=month, year=year, color=color, avg=avg)
    )

    return year, source

In [ ]:
def all_years():
    years = set(year)
    start_year = min(years)
    end_year = max(years)

    return [str(y) for y in range(int(start_year), int(end_year), 5)]

In [ ]:
def plot(year, source):
    fig = bkh_plt.figure(title="De Bilt, NL Temperature (1901 - 2014)",
                         x_range=all_years(), 
                         y_range=list(reversed(ts.short_months())),
                         toolbar_location="left", 
                         tools="resize,hover,save,pan,box_zoom,wheel_zoom")

    fig.rect("year", "month", 1, 1, source=source,
        color="color", line_color=None)

    fig.xaxis.major_label_orientation = np.pi/3

    hover = fig.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ('date', '@month @year'),
        ('avg', '@avg'),
    ])

    bkh_plt.output_notebook()
    bkh_plt.show(fig)
    

In [ ]:
year, source = create_source()
plot(year, source)